In [2]:
import logging
import json
import torch
from types import SimpleNamespace
from vilbert.vilbert import VILBertActionGrounding, BertConfig
from pytorch_transformers.tokenization_bert import BertTokenizer
from pytorch_transformers.optimization import AdamW, WarmupLinearSchedule
import torch.distributed as dist
from VLN_config import config as args
import random
import pandas as pd
from dataLoader import DataLoader

In [3]:
import sys
import os
import torch
import yaml

import numpy as np
import matplotlib.pyplot as plt
import PIL

from PIL import Image
import cv2
import argparse
import glob
import pdb

import torchvision.models as models
import torchvision.transforms as transforms

from faster_rcnn import feature_extractor_new as f_extractor
from faster_rcnn.feature_extractor_new import featureExtractor
#%matplotlib inline  

In [3]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)
print(args)

namespace(adam_epsilon=1e-08, baseline=False, bert_model='bert-base-uncased', best_features=5, clean_train_sets=True, config_file='config/bert_base_6layer_6conect.json', distributed=False, do_lower_case=True, dynamic_attention=False, from_pretrained='save/multitask_model/multi_task_model.bin', gradient_accumulation_steps=1, img_weight=1, in_memory=False, learning_rate=0.0001, local_rank=-1, max_temporal_memory_buffer=3, mean_layer=False, num_key_frames=6, num_train_epochs=10.0, num_workers=0, objective=1, predict_feature=False, save_name='', seed=42, split='mteval', start_epoch=0, task_specific_tokens=True, tasks='1', threshold_similarity=0.7, track_temporal_features=True, train_batch_size=1, visual_target=0, warmup_proportion=0.1, without_coattention=False)


In [4]:
tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

07/23/2020 15:35:36 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/salem/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [14]:
config = BertConfig.from_json_file(args.config_file)
bert_weight_name = json.load(
    open("config/" + args.bert_model + "_weight_name.json", "r")
)

tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

config.track_temporal_features = args.track_temporal_features
config.mean_layer = args.mean_layer
config.max_temporal_memory_buffer = args.max_temporal_memory_buffer

model = VILBertActionGrounding.from_pretrained(
    args.from_pretrained, config=config, default_gpu=True
)

07/23/2020 16:31:01 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/salem/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
07/23/2020 16:31:01 - INFO - vilbert.utils -   loading weights file save/multitask_model/multi_task_model.bin
07/23/2020 16:32:08 - INFO - vilbert.utils -   Weights of VILBertActionGrounding not initialized from pretrained model: ['positional_enc.weight', 'positional_enc.bias', 'img_emb_mean.weight', 'img_emb_mean.bias', 'action_cls.predictions.bias', 'action_cls.predictions.transform.dense.weight', 'action_cls.predictions.transform.dense.bias', 'action_cls.predictions.transform.LayerNorm.weight', 'action_cls.predictions.transform.LayerNorm.bias', 'action_cls.predictions.decoder.weight', 'action_cls.bi_seq_relationship.weight', 'action_

In [4]:
# load data
frcnn_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
data_loader = DataLoader("json_data.json", frcnn_model, save_or_not = False)
path = 'DataLoader.pt'
data_loaded = data_loader.load_dataloader(path)

In [5]:
data_loaded[-2]

[[{'bbox': array([[1.05088104e+02, 4.40459938e+01, 1.75131607e+02, 6.58351440e+01],
          [1.72085892e+02, 4.40263405e+01, 2.37832245e+02, 6.53740082e+01],
          [1.11023499e+02, 4.36034279e+01, 2.37218262e+02, 6.63305206e+01],
          [0.00000000e+00, 6.23910141e+00, 2.57281437e+01, 8.47814178e+01],
          [1.46527290e-02, 6.88947773e+00, 2.65441303e+01, 8.65501328e+01],
          [1.04400116e+02, 1.20458722e+00, 2.38441360e+02, 2.47956734e+01],
          [1.12575531e-01, 1.09584824e+02, 3.53457298e+01, 1.90196045e+02],
          [9.44248581e+01, 0.00000000e+00, 2.44965912e+02, 2.74502563e+01],
          [1.12575531e-01, 1.09584824e+02, 3.53457298e+01, 1.90196045e+02],
          [9.44248581e+01, 0.00000000e+00, 2.44965912e+02, 2.74502563e+01],
          [6.74335938e+01, 3.34082108e+01, 2.88827332e+02, 8.51654358e+01],
          [8.92899017e+01, 3.52115364e+01, 1.75789886e+02, 7.81162872e+01],
          [1.85623474e+02, 3.60988235e+01, 2.62392395e+02, 7.17223969e+01],
    

In [23]:
def split_train_val (data_loaded, split_portion = 0.9):
    # WE are going to ignore infos
    features_masked, pos_enc, spatial, image_mask, tokenized_text, masked_text, masked_lm_token, input_mask, segment_ids, co_attention_mask, infos, masked_img_labels  = data_loaded
    
    indexes = list(range(features_masked.shape[0]))
    random.shuffle(indexes)
    split_train = int(features_masked.shape[0] * split_portion)
    #split_val = features_masked.shape[0] - split_train
    indexes_train = indexes[:split_train]
    indexes_val = indexes[split_train:]
    data_loaded_train = (features_masked[indexes_train], 
                         pos_enc[indexes_train], 
                         spatial[indexes_train], 
                         image_mask[indexes_train], 
                         tokenized_text[indexes_train], 
                         masked_text[indexes_train], 
                         masked_lm_token[indexes_train], 
                         input_mask[indexes_train], 
                         segment_ids[indexes_train], 
                         co_attention_mask[indexes_train],  
                         masked_img_labels[indexes_train])
    data_loaded_val = (features_masked[indexes_val], 
                         pos_enc[indexes_val], 
                         spatial[indexes_val], 
                         image_mask[indexes_val], 
                         tokenized_text[indexes_val], 
                         masked_text[indexes_val], 
                         masked_lm_token[indexes_val], 
                         input_mask[indexes_val], 
                         segment_ids[indexes_val], 
                         co_attention_mask[indexes_val],  
                         masked_img_labels[indexes_val])
    return data_loaded_train, data_loaded_val 

In [16]:
print(features_masked.shape)
print(pos_enc.shape)
print(spatial.shape)
print(tokenized_text.shape)
print(masked_text.shape)
print(masked_lm_token.shape)
print(input_mask.shape)
print(segment_ids.shape)
print(co_attention_mask.shape)
print(len(infos))
print(masked_img_labels.shape)
print(masked_lm_token.shape)

torch.Size([3, 31, 2048])
torch.Size([3, 31, 7])
torch.Size([3, 31, 5])
torch.Size([3, 37])
torch.Size([3, 37])
torch.Size([3, 37])
torch.Size([3, 37])
torch.Size([3, 37])
torch.Size([3, 31, 37])
3
torch.Size([3, 31])
torch.Size([3, 37])


In [15]:
model.cpu()
model.train()
optimizer = AdamW(model.parameters(),
                    lr=args.learning_rate,
                    eps=args.adam_epsilon,
                    betas=(0.9, 0.98),)



In [19]:
print(co_attention_mask.unsqueeze(0).cpu().shape)
print(image_mask.unsqueeze(0).cpu().shape)
print(input_mask.unsqueeze(0).cpu().shape)
print(segment_ids)

torch.Size([1, 3, 31, 37])
torch.Size([1, 3, 31])
torch.Size([1, 3, 37])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [20]:
for key, value in dict(model.named_parameters()).items():
    if not value.requires_grad:
        print("This parameter does have grad", key)

In [24]:
batch_size = args.train_batch_size 
data_train , data_val = split_train_val(data_loaded)
(features_masked_train, pos_enc_train, spatial_train, image_mask_train, tokenized_text_train, masked_text_train,
     masked_lm_token_train, input_mask_train, 
    segment_ids_train, co_attention_mask_train, masked_img_labels_train)  = data_train

(features_masked_val, pos_enc_val, spatial_val, image_mask_val, tokenized_text_val, masked_text_val,
     masked_lm_token_val, input_mask_val, 
    segment_ids_val, co_attention_mask_val, masked_img_labels_val)  = data_val
loss_result_csv = pd.DataFrame(columns = ['epochs', 'train_loss', 'val_loss'])

In [ ]:
for epoch in range(2):
    i = 0
    loss_train_cum = 0
    while (i < data_train[0].shape[0]):
        pred_t_train, pred_v_train, att_train = model(input_ids = masked_text_train[i:i+batch_size].cpu(),
                                image_feat = features_masked_train.cpu(), # Linear(2048*config.max_temporal_memory_buffer, 2048)
                                image_loc = spatial_train.cpu(),  #Linear(in_features=5, out_features=1024, bias=True)
                                image_pos_input = pos_enc_train.cpu(),   #Linear(7, 2048)/(6, 2048)
                                token_type_ids = segment_ids_train.cpu(), 
                                attention_mask = input_mask_train.cpu(), 
                                image_attention_mask = image_mask_train.cpu(),
                                output_all_attention_masks=True)
        i = i+batch_size
        #print(len(att[0]), len(att[1]), len(att[2]))
        #print(att)
        optimizer.zero_grad()
        # target text: masked_lm_token - targetimage: masked_img_labels
        masked_lm_loss_train = model.lang_criterion(pred_t_train.view(-1, 30522), masked_lm_token_train.cpu().view(-1))
        img_loss_train = model.vis_criterion(pred_v_train.view(-1, 91), masked_img_labels_train.view(-1).cpu()) # why dim 2 (to check) 
        loss_train = masked_lm_loss_train + img_loss_train
        loss_train.backward()
        loss_train_cum += loss_train
        optimizer.step()
    loss_train_cum = loss_train_cum/data_train[0].shape[0]
    print("epoch: " , epoch, " Train loss: ", loss_train_cum)
    # Validation
    pred_t_val, pred_v_val, att_val = model(input_ids = masked_text_val.cpu(),
                            image_feat = features_masked_val.cpu(), # Linear(2048*config.max_temporal_memory_buffer, 2048)
                            image_loc = spatial_val.cpu(),  #Linear(in_features=5, out_features=1024, bias=True)
                            image_pos_input = pos_enc_val.cpu(),   #Linear(7, 2048)/(6, 2048)
                            token_type_ids = segment_ids_val.cpu(), 
                            attention_mask = input_mask_val.cpu(), 
                            image_attention_mask = image_mask_val.cpu(),
                            output_all_attention_masks=True)
    #print(len(att[0]), len(att[1]), len(att[2]))
    #print(att)
    optimizer.zero_grad()
    # target text: masked_lm_token - targetimage: masked_img_labels
    masked_lm_loss_val = model.lang_criterion(pred_t_val.view(-1, 30522), masked_lm_token_val.cpu().view(-1))
    img_loss_val = model.vis_criterion(pred_v_val.view(-1, 91), masked_img_labels_val.view(-1).cpu()) # why dim 2 (to check) 
    loss_val = masked_lm_loss_val + img_loss_val
    loss_val = loss_val / data_val[0].shape[0]
    print("epoch: " , epoch," Val loss: ", loss_val)
    loss_result_csv = loss_result_csv.append(pd.DataFrame(epoch, loss_train_cum, loss_val))
    if i % 1 == 0:
        loss_result_csv.to_csv('epoch_loss.csv')
        model.save_pretrained('vilber_action_grounding.bin')

In [ ]:
loss_masked_lang = model.lang_criterion(pred_t, labels_t)


In [21]:
print((att))

([None, None, None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None], [None, None, None, None, None, None])


In [ ]:
features = torch.stack(feature_list, dim=0).float().cuda()
    spatials = torch.stack(image_location_list, dim=0).float().cuda()
    image_mask = torch.stack(image_mask_list, dim=0).byte().cuda()
    co_attention_mask = torch.zeros((num_image, num_boxes, max_length)).cuda()

    prediction(text, features, spatials, segment_ids, input_mask, image_mask, co_attention_mask, task)

In [ ]:
print(data[1]["imgs"]["feat"].shape)
concate = torch.cat((data[1]["imgs"]["feat"].unsqueeze(0), data[0]["imgs"]["feat"].unsqueeze(0)), dim=0)
print(concate.shape)

In [ ]:
print(data)

In [ ]:
print("Structure of data: ")
print(" · Number of instructions: ", len(data))
print("  · Per instruction we have: ", data[0].keys())
print("    · The 'desc' of the instruction has: ", data[0]["desc"].keys())
print("       - lists of length tokenized text -->",len(data[0]["desc"]["tokenized_text"]))
print("    · The 'imgs' of the instruction has:", data[0]["imgs"].keys())
for k, v in data[0]["imgs"].items():
    print(k, v)
    #print(len(data[0]["imgs"]["spatial"][0]))

In [ ]:
model

In [ ]:
for key, value in dict(model.named_parameters()).items():
    if not value.requires_grad:
        print("This parameter does have grad", key)

In [ ]:
model.eval()
cuda = torch.cuda.is_available()
if cuda: model = model.cuda(0)
#Why do we initialize Tokenizer again?
tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

## Data preparation

In [ ]:
def bert_tokenize( text):
    text = '[CLS]' + text + '[SEP]'
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    return indexed_tokens

In [ ]:
text = "swimming elephant"
indexed_tokens = bert_tokenize(text)
#indexed_tokens, modified_indexed_tokens, masked_lm_labels = pretask_mask_lang_tokens(indexed_tokens)

query = '[CLS]' + text + '[SEP]'
tokens = tokenizer.encode(query)
print(tokens)

In [ ]:
def prediction(question, features, spatials, segment_ids, input_mask, image_mask, co_attention_mask, task_tokens, ):
    print('input question size question: ', question.shape)
    pos_enc_input = torch.FloatTensor([0,1,2,3,4,5])
    print(question.shape)
    masked_lm_loss, masked_img_loss, prediction_t, prediction_v, all_attention_mask = model(
        input_ids=question, image_feat=features, image_loc=spatials,image_pos_input=pos_enc_input, output_all_attention_masks=True
    )
    return masked_lm_loss, masked_img_loss, prediction_t, prediction_v, all_attention_mask

In [ ]:
def custom_prediction(query, task, features, infos):

#     print(query)
#     query = '[CLS]' + query + '[SEP]'
#     tokens = tokenizer.encode(query)
#     print(tokens)
    #tokens= 
    #tokens = tokenizer.add_special_tokens_single_sentence(tokens)
    
    tokens = bert_tokenize(query)
    segment_ids = [0] * len(tokens)
    input_mask = [1] * len(tokens)

    max_length = 37
    if len(tokens) < max_length:
        # Note here we pad in front of the sentence
        padding = [0] * (max_length - len(tokens))
        tokens = tokens + padding
        input_mask += padding
        segment_ids += padding

    text = torch.from_numpy(np.array(tokens)).cuda().unsqueeze(0)
    input_mask = torch.from_numpy(np.array(input_mask)).cuda().unsqueeze(0)
    segment_ids = torch.from_numpy(np.array(segment_ids)).cuda().unsqueeze(0)
    task = torch.from_numpy(np.array(task)).cuda().unsqueeze(0)

    num_image = len(infos)

    feature_list = []
    image_location_list = []
    image_mask_list = []
    for i in range(num_image):
        image_w = infos[i]['image_width']
        image_h = infos[i]['image_height']
        feature = features[i]
        num_boxes = feature.shape[0] #first dim size = number boxes

        g_feat = torch.sum(feature, dim=0) / num_boxes # Mean of features of all the selected regions
        num_boxes = num_boxes + 1
        feature = torch.cat([g_feat.view(1,-1), feature], dim=0)
        boxes = infos[i]['bbox']
        image_location = np.zeros((boxes.shape[0], 5), dtype=np.float32)
        image_location[:,:4] = boxes
        image_location[:,4] = (image_location[:,3] - image_location[:,1]) * (image_location[:,2] - image_location[:,0]) / (float(image_w) * float(image_h))
        image_location[:,0] = image_location[:,0] / float(image_w)
        image_location[:,1] = image_location[:,1] / float(image_h)
        image_location[:,2] = image_location[:,2] / float(image_w)
        image_location[:,3] = image_location[:,3] / float(image_h)
        g_location = np.array([0,0,1,1,1])
        image_location = np.concatenate([np.expand_dims(g_location, axis=0), image_location], axis=0)
        image_mask = [1] * (int(num_boxes))

        feature_list.append(feature)
        image_location_list.append(torch.tensor(image_location))
        image_mask_list.append(torch.tensor(image_mask))

    features = torch.stack(feature_list, dim=0).float().cuda()
    spatials = torch.stack(image_location_list, dim=0).float().cuda()
    image_mask = torch.stack(image_mask_list, dim=0).byte().cuda()
    co_attention_mask = torch.zeros((num_image, num_boxes, max_length)).cuda()
#     print("text: ", text.shape)
#     print("feat: ", features.shape)
#     print("spatials: ", spatials.shape)
#     print("segments_id: ", segment_ids)
#     print("input_mask: ", input_mask)
#     print("image_mask: ", image_mask)
#     print("coatenttion_mask: ", co_attention_mask)
    return prediction(text, features, spatials, segment_ids, input_mask, image_mask, co_attention_mask, task)

In [ ]:
frcnn_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# read image 
#pic = "faster_rcnn/test2.png"
#pic1 = "faster_rcnn/test.png"
#image_paths = [pic, pic1]



image_path = ['demo/1.jpg']
features, pos_enc, infos = featureExtractor(image_path, frcnn_model).extract_features()
print("features: ", type(features))
print("infos: ", infos)
#features, infos = f_extractor.extract_features(image_path, frcnn_model)


img = PIL.Image.open(image_path[0]).convert('RGB')
img = torch.tensor(np.array(img))

plt.axis('off')
plt.imshow(img)
plt.show()
    
query = "swimming elephant"
task = [9]
masked_lm_loss, masked_img_loss, prediction_t, prediction_v, all_attention_mask = custom_prediction(query, task, features, infos)

In [ ]:
print(data[0]["imgs"]["pos_enco"])

In [ ]:
masked_lm_loss

In [ ]:
 prediction_t.shape

In [ ]:
prediction_t

In [ ]:
prediction_v.shape

In [ ]:
prediction_v

In [ ]:
all_attention_mask

In [ ]:
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
config.visual_target = args.visual_target

optimizer_grouped_parameters = []
for key, value in dict(model.named_parameters()).items():
    if value.requires_grad:
        if "cls" in key:
            lr = args.learning_rate
        else:
            lr = args.learning_rate * 0.1
        if any(nd in key for nd in no_decay): # No decay
            optimizer_grouped_parameters += [
                {"params": [value],
                 "lr": lr,
                 "weight_decay": 0.0}
            ]
        
        elif not any(nd in key for nd in no_decay):
            optimizer_grouped_parameters += [
                {"params": [value],
                 "lr": lr,
                 "weight_decay": 0.01}
            ]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=args.learning_rate,
    eps=args.adam_epsilon,
    betas=(0.9, 0.98),
)

num_dataset_points = 19

num_train_optimization_steps = int(
    num_dataset_points
    / args.train_batch_size
    / args.gradient_accumulation_steps
) * (args.num_train_epochs - args.start_epoch)

scheduler = WarmupLinearSchedule(
    optimizer,
    warmup_steps=args.warmup_proportion * num_train_optimization_steps,
    t_total=num_train_optimization_steps,
)

In [ ]:
# model.cuda()
# for state in optimizer.state.values():
#     for k, v in state.items():
#         if torch.is_tensor(v):
#             state[k] = v.cuda()

logger.info("***** Running training *****")
logger.info("Num examples = %d", num_dataset_points)
logger.info("Batch size = %d", args.train_batch_size)
logger.info("Num steps = %d", num_train_optimization_steps)

In [ ]:
startIterID = 0
global_step = 0

for epoch in range(int(args.start_epoch), int(args.num_train_epochs)):
    model.train()
    for step, batch in enumerate(train_dataset):
        iterId = startIterID + step + (epochId * len(train_dataset))
    